In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from matplotlib.colors import ListedColormap
from sklearn import svm, neighbors, tree, ensemble, model_selection, metrics
from sklearn.inspection import DecisionBoundaryDisplay
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

In [5]:
data_folder = "data/traces/"

def read_date(date: str):
    """Aggregate data of a single from parts"""
    data = pd.DataFrame()

    for i in range(50):
        try:
            frame = pd.read_csv(data_folder + date + "-{}.csv".format(i))
            data = pd.concat([data, frame], axis=0)
        except OSError:
            pass

    return data


def read_date_part(date_part: str):
    """Read data of a part of a capture date"""
    data = pd.DataFrame()

    try:
        frame = pd.read_csv(data_folder + date_part + ".csv")
        data = pd.concat([data, frame], axis=0)
    except OSError:
        pass

    return data

data = read_date_part("16-09-23")
data.set_index("timestamp")
print(data)

       packet_id            timestamp  packet_size            eth_src   
0              0  1474552802257569024           70  18:b7:9e:02:20:44  \
1              1  1474552802257692160           66  18:b7:9e:02:20:44   
2              2  1474552802323085056           66  14:cc:20:51:33:ea   
3              3  1474552802383173888           60  d0:52:a8:00:67:5e   
4              5  1474552802447910912           88  70:ee:50:18:34:43   
...          ...                  ...          ...                ...   
46193      49996  1474560343348738048           66  14:cc:20:51:33:ea   
46194      49997  1474560343487133952           66  18:b7:9e:02:20:44   
46195      49998  1474560343508438016           42  ec:1a:59:83:28:11   
46196      49999  1474560343508498944           42  14:cc:20:51:33:ea   
46197      50000  1474560343931791104          156  30:8c:fb:2f:e4:b2   

       device_name protocol    iot  
0              NaN      Raw  False  
1              NaN      TCP  False  
2           

In [6]:
def attach_window_id(timestamped_frame: pd.DataFrame) -> list[pd.DataFrame]:
    """Slice the dataframe into 1-second intervals"""
    first_packet_time = timestamped_frame.iloc[0]['timestamp']
    last_packet_time = timestamped_frame.iloc[-1]['timestamp']
    windowed_frame = timestamped_frame.set_index('timestamp', drop=False) 

    ns_in_sec = int(1e9)
    number_of_windows = int((last_packet_time - first_packet_time) / ns_in_sec) + 1

    try:
        windowed_frame.insert(0, value=0, column="window_id")
    except Exception as e:
        print("It seems like the column 'window_id' already exists. Just updating values...")
        print(e)
    finally:
        windowed_frame.window_id.astype(np.int64)

    for window_id in range(0, number_of_windows):
        from_time = first_packet_time + window_id * ns_in_sec
        to_time = first_packet_time + (window_id  + 1) * ns_in_sec
        windowed_frame.loc[from_time:to_time, 'window_id'] = window_id
    
    windowed_frame.set_index('window_id', inplace=True)
    return windowed_frame


devices = pd.read_csv("data/list_of_devices.csv")
windowed_frame = attach_window_id(data)
print(windowed_frame)

           packet_id            timestamp  packet_size            eth_src   
window_id                                                                   
0                  0  1474552802257569024           70  18:b7:9e:02:20:44  \
0                  1  1474552802257692160           66  18:b7:9e:02:20:44   
0                  2  1474552802323085056           66  14:cc:20:51:33:ea   
0                  3  1474552802383173888           60  d0:52:a8:00:67:5e   
0                  5  1474552802447910912           88  70:ee:50:18:34:43   
...              ...                  ...          ...                ...   
7541           49996  1474560343348738048           66  14:cc:20:51:33:ea   
7541           49997  1474560343487133952           66  18:b7:9e:02:20:44   
7541           49998  1474560343508438016           42  ec:1a:59:83:28:11   
7541           49999  1474560343508498944           42  14:cc:20:51:33:ea   
7541           50000  1474560343931791104          156  30:8c:fb:2f:e4:b2   

In [8]:
def generate_features_labeled(windowed_frame: pd.DataFrame, device_frame: pd.DataFrame):
    """Calculate the mode, mean, and median of a window"""
    features = pd.DataFrame()

    for device_index in device_frame.index:
        device = device_frame.iloc[device_index]
        device_packets = windowed_frame.loc[windowed_frame['eth_src'] == device['eth_src']]
        device_features = generate_features(device_packets)

        device_features.insert(len(device_features.columns), "device_name", device['device_name'])
        device_features.insert(len(device_features.columns), "iot", device['iot'])

        features = pd.concat([features, device_features])

    features.columns = ["window_id", "mean", "std", "n_bytes", "device_name", "iot"]
    features.set_index("window_id", inplace=True)

    return features

def generate_features(windowed_frame: pd.DataFrame):
    features =  pd.DataFrame()

    for window_id in windowed_frame.index.unique():
        packet_sizes = windowed_frame.loc[window_id, 'packet_size']

        mean = packet_sizes.mean()
        std = packet_sizes.std()
        n_bytes = packet_sizes.sum()

        features = pd.concat([features, pd.DataFrame([[window_id, mean, std, n_bytes]])], axis=0)

    return features

def get_devices_in_window(windowed_frame: pd.DataFrame):
    devices_in_window = pd.DataFrame()
    
    for window_id in windowed_frame.index.unique():
        devices = windowed_frame.loc[window_id]['device_name']
        # iot = windowed_frame.loc[window_id]['iot']

        if not isinstance(devices, str):
            devices = devices.unique() 
            
        devices_in_window = pd.concat([devices_in_window, pd.DataFrame({"window_id": window_id, "devices": devices})])

    return devices_in_window

labeled_features = generate_features_labeled(windowed_frame, devices)

In [10]:
def train_and_test_classifiers_under_over(labeled_features, target_label, random_state=None):
    """
    Trains and tests a series of models on the given, labeled data
    (Knn, RF, DT, SVM, Majority Voting)
    """
    X = labeled_features.iloc[:, 1:3]
    y = labeled_features.iloc[:][target_label]

    oversampling = SMOTE(random_state=random_state)
    # Resource [32] is actually really old (2009), NearMiss is a better alternative than RUS
    undersampling = NearMiss()

    # first oversample, then undersample
    X, y = oversampling.fit_resample(X, y)
    X, y = undersampling.fit_resample(X, y)

    models = {
        "Knn" : neighbors.KNeighborsClassifier(n_neighbors=5, weights="distance"),
        # "SVM" : svm.SVC(),
        "DT"  : tree.DecisionTreeClassifier(),
        "RF"  : ensemble.RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
    }
    # I don't really see the advantage of using an ensemble method in conjuction with the
    # under- and oversample techniques. I can't find any indication in the paper the referenced [32]
    # that it actually works. So, I'm just keeping this here.
    models["Majority Voting"] = ensemble.VotingClassifier(list(models.items()), voting='hard')

    trained = {}

    for (name, model) in models.items():
        trained[name] = model.fit(X, y)
        accuracy = metrics.accuracy_score(y, model.predict(X))

        print("{} mean accuracy with SMOTE balancing: {}".format(name, accuracy))
        
    return trained
    

def train_and_test_classifiers_stratified(labeled_features, target_label, n_splits=10, shuffle=False, random_state=None):
    """
    Trains and tests a series of models on the given, labeled data
    (Knn, RF, DT, SVM, Majority Voting)
    """

    X = labeled_features.iloc[:, 1:3]
    y = labeled_features.iloc[:][target_label]
    kf = model_selection.StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    models = {
        "Knn" : neighbors.KNeighborsClassifier(n_neighbors=5, weights="distance"),
        # "SVM" : svm.SVC(),
        "DT"  : tree.DecisionTreeClassifier(),
        "RF"  : ensemble.RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
    }
    models["Majority Voting"] = ensemble.VotingClassifier(list(models.items()), voting='hard')

    trained = {}

    print("Training and testing models on {}".format(target_label))
    for (name, model) in models.items():
        accuracy = 0.0
        
        for (train_index, test_index) in kf.split(X, y):
            trained[name] = model.fit(X.iloc[train_index], y.iloc[train_index])
            accuracy += metrics.accuracy_score(y.iloc[test_index], model.predict(X.iloc[test_index]))

        print("{} mean accuracy over {} splits: {}".format(name, n_splits, accuracy/n_splits))
        
    return trained

iot_classfiers = train_and_test_classifiers_under_over(labeled_features, 'iot')
device_classifiers = train_and_test_classifiers_stratified(labeled_features.loc[labeled_features['iot'] == True], 'device_name')

iot_classfiers
device_classifiers

Knn mean accuracy with SMOTE balancing: 0.8105410261529248
DT mean accuracy with SMOTE balancing: 0.9033073800492447
RF mean accuracy with SMOTE balancing: 0.9025420909030412
Majority Voting mean accuracy with SMOTE balancing: 0.9033073800492447
Training and testing models on device_name
Knn mean accuracy over 10 splits: 0.7250934881236195
DT mean accuracy over 10 splits: 0.8168615277212995
RF mean accuracy over 10 splits: 0.8183258870629799
Majority Voting mean accuracy over 10 splits: 0.817327572994269


{'Knn': KNeighborsClassifier(weights='distance'),
 'DT': DecisionTreeClassifier(),
 'RF': RandomForestClassifier(n_estimators=10, random_state=0),
 'Majority Voting': VotingClassifier(estimators=[('Knn', KNeighborsClassifier(weights='distance')),
                              ('DT', DecisionTreeClassifier()),
                              ('RF',
                               RandomForestClassifier(n_estimators=10,
                                                      random_state=0))])}

In [ ]:
from sklearn import metrics

def test_realistic(iot_classifier, device_classifier, unlabeled_features, devices_in_window):
    assert len(unlabeled_features) == len(devices_in_window)

    # accuracy = metrics.accuracy_score(devices_in_window, iot_classifier.predict(unlabeled_features))
    accuracy = metrics.accuracy_score(devices_in_window, device_classifier.predict(unlabeled_features))
    print("Accuracy:", accuracy)
 

unlabeled_features = generate_features(windowed_frame)
devices_in_window = get_devices_in_window(windowed_frame)

test_realistic(iot_classifier, device_classifier, unlabeled_features, devices_in_window)

ValueError: If using all scalar values, you must pass an index